# 1. Đọc dữ liệu

In [26]:
import pandas as pd
import numpy as np
import os

# --- 1. LOAD DỮ LIỆU ---
print("Đang đọc dữ liệu từ file CSV...")
path = './' 

orders = pd.read_csv(path + 'olist_orders_dataset.csv')
items = pd.read_csv(path + 'olist_order_items_dataset.csv')
products = pd.read_csv(path + 'olist_products_dataset.csv')
payments = pd.read_csv(path + 'olist_order_payments_dataset.csv')
reviews = pd.read_csv(path + 'olist_order_reviews_dataset.csv')
customers = pd.read_csv(path + 'olist_customers_dataset.csv')
sellers = pd.read_csv(path + 'olist_sellers_dataset.csv')
translation = pd.read_csv(path + 'product_category_name_translation.csv')

print("Đã đọc xong! Bắt đầu xử lý...")


Đang đọc dữ liệu từ file CSV...
Đã đọc xong! Bắt đầu xử lý...


# 2. Dịch tên sản phẩm

In [27]:
# Nối bảng Product với bảng Translation để lấy tên tiếng Anh
products = pd.merge(products, translation, on='product_category_name', how='left')

# Chỉ giữ lại cột tên tiếng Anh, bỏ tên Bồ Đào Nha 
products['product_category_name'] = products['product_category_name_english']
products.drop(columns=['product_category_name_english'], inplace=True)

# 3. Nối bảng

In [28]:
# Nguyên tắc: Lấy bảng 'orders' làm gốc, nối các bảng khác vào

# Bước 3.1: Nối chi tiết sản phẩm (Items)
# Lưu ý: Một đơn hàng có thể có nhiều sản phẩm -> Số dòng sẽ tăng lên
df = pd.merge(orders, items, on='order_id', how='left')

# Bước 3.2: Nối thông tin Sản phẩm (Products)
df = pd.merge(df, products, on='product_id', how='left')

# Bước 3.3: Nối thông tin Khách hàng (Customers)
# Để lấy customer_unique_id (dùng cho Gom cụm) và customer_state (dùng cho Map)
df = pd.merge(df, customers, on='customer_id', how='left')

# Bước 3.4: Nối thông tin Đánh giá (Reviews)
df = pd.merge(df, reviews, on='order_id', how='left')

# 4. Làm sạch dữ liệu 

In [29]:
# 4.1. Chuyển đổi định dạng ngày tháng (Object -> Datetime)
date_cols = ['order_purchase_timestamp', 'order_approved_at', 
             'order_delivered_carrier_date', 'order_delivered_customer_date', 
             'order_estimated_delivery_date']

for col in date_cols:
    df[col] = pd.to_datetime(df[col])

In [30]:
# 4.2. Xóa dữ liệu trùng lặp
df = df.drop_duplicates()

In [31]:
#Cột chỉ có 1 giá trị
single_val_cols = [col for col in df.columns if df[col].nunique() <= 1]
print(f"Các cột vô nghĩa (chỉ có 1 giá trị): {single_val_cols}")

df.drop(columns=single_val_cols, inplace=True)

Các cột vô nghĩa (chỉ có 1 giá trị): []


In [32]:
#Cột thiếu
null_percent = df.isnull().mean() * 100
cols_to_drop = null_percent[null_percent > 90].index.tolist() # Ngưỡng 90%
print(f" Các cột quá nhiều Null (>90%): {cols_to_drop}")

# Hành động: Xóa cột rác
df.drop(columns=cols_to_drop, inplace=True)

# Xóa dòng lỗi (Ví dụ thiếu ID sản phẩm)
df.dropna(subset=['product_id', 'order_id'], inplace=True)

 Các cột quá nhiều Null (>90%): []


In [33]:
# 4.3. Xóa các dòng đơn hàng bị hủy hoặc chưa giao (vì không phân tích được Review)
df = df.dropna(subset=['order_delivered_customer_date', 'review_score'])

In [34]:
# 4.5. Xóa giá tiền âm (Lỗi hệ thống)
df = df[df['price'] >= 0]

# 5. Tạo biến

In [35]:
# Tính thời gian giao hàng thực tế (Actual Delivery Days)
df['delivery_days'] = (df['order_delivered_customer_date'] - df['order_purchase_timestamp']).dt.days

In [36]:
# Tạo nhãn Giao trễ (Is Late)
df['is_late'] = np.where(df['order_delivered_customer_date'] > df['order_estimated_delivery_date'], 1, 0)

In [37]:
# Tạo nhãn Review Tốt/Xấu (Binary Classification)
# 1: Tốt (4-5 sao), 0: Xấu (1-3 sao)
# Xử lý Null Review thành -1 để không lỗi
df['review_score'] = df['review_score'].fillna(-1)
df['review_binary'] = np.where(df['review_score'] >= 4, 1, 0)

# 6. Kiểm tra & lưu file

In [39]:
# KẾT THÚC & XUẤT FILE
# ==============================================================================
# Lọc bỏ các dòng rác (không có sản phẩm)
df_final = df.dropna(subset=['product_id'])

# Lưu ra file Master
df.to_csv('master_data.csv', index=False)
print("XONG! File 'master_data.csv' đã được tạo thành công.")

XONG! File 'master_data.csv' đã được tạo thành công.
